In [19]:
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

x = load_iris().data
y = load_iris().target

In [20]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=29)

x_train.shape

(112, 4)

In [72]:
def learning_schedule(t):
    return 5 / (t + 50)

def SGD(X, y, n_epoch, eta):
    theta = np.random.rand(X.shape[1], 1)
    print(theta)
    m = X.shape[0]
    for epoch in range(1, n_epoch):
        for n in range(m):
            r = np.random.randint(0, m-1)
            xi, yi = X[r:r+1], y[r:r+1]
            diff = xi.T @ ((xi @ theta) - yi)
            eta = learning_schedule(epoch * m + n)
            theta = theta - (diff * eta)
    return theta

theta = SGD(x_train, y_train, 50, 0.1)

print(theta)

preds = x_test @ theta
preds = np.rint(preds)
print(f'SGD Acc: {accuracy_score(y_test, preds)}')

[[ 0.0959091 ]
 [ 0.09855633]
 [ 0.4219622 ]
 [ 0.53794816]]
[[-0.07158914]
 [-0.05177736]
 [ 0.28547318]
 [ 0.43067766]]
SGD Acc: 0.9736842105263158


In [63]:
def normal(X):
    return X / X.max(axis=0)

In [64]:
def predict(X, theta):
    return np.c_[np.ones(len(X)), normal(X)] @ theta

In [66]:
def BGD(X, y, n_iter, eta):
    X = np.c_[np.ones(len(X)), normal(X)]
    y = np.array([y.T])
    theta = np.random.rand(1, X.shape[1])
    for i in range(n_iter):
        for c in range(X.shape[1]):
            nX = X[:,c]
            nT = theta[:,c]
            gradients = ((X @ theta.T) - y.T).T @ nX
            theta[:,c] = nT - eta * gradients
    return theta.T

theta = BGD(x_train, y_train, 1000, 0.001)

print(theta)

preds = predict(x_test, theta)
preds = np.rint(preds)
print(f'\nBGD Acc: {accuracy_score(y_test, preds)}')

[[-0.3271177 ]
 [ 0.56682146]
 [-0.54685397]
 [ 1.02323173]
 [ 1.49379723]]

BGD Acc: 0.9210526315789473


In [67]:
def MBGD(X, y, n_epoch, batch_size, eta):
    X = np.c_[np.ones(len(X)), normal(X)]
    y = np.array([y.T])
    theta = np.random.rand(1, X.shape[1])
    m = X.shape[0]
    for epoch in range(n_epoch):
        shuffi = np.random.permutation(m)
        xs, ys = X[shuffi], y.T[shuffi]
        for i in range(0, m, batch_size):
            for c in range(X.shape[1]):
                r = np.random.randint(0, m-1)
                xi, yi = X[i:i+batch_size], y.T[i:i+batch_size]
                gradients = ((xi @ theta.T) - yi).T @ xi[:,c]
                theta[:,c] = theta[:,c] - (gradients * eta)
    return theta.T
        
theta = MBGD(x_train, y_train, 100, 20, 0.01)

print(theta)

preds = predict(x_test, theta)
preds = np.rint(preds)
print(f'\n MBGD Acc: {accuracy_score(y_test, preds)}')

[[-0.26815798]
 [ 0.2676647 ]
 [-0.4017181 ]
 [ 1.03258355]
 [ 1.59638522]]

 MBGD Acc: 0.9473684210526315
